# Training main:

take from cyanite notebook the interface to observe tags distribution and create datasets


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
from torch.autograd import Variable

from frontend import Frontend_mine, Frontend_won
from backend import Backend, Backend2
from data_loader import get_DataLoader

main_dict = {"frontend_dict":
             {"list_out_channels":[128,128,128,256,256,256], 
              "list_kernel_sizes":[(3,3),(3,3),(3,3),(3,3),(3,3),(3,3)],
              "list_pool_sizes":  [(3,2),(2,2),(2,2),(2,1),(2,1),(2,1)], 
              "list_avgpool_flags":[False,False,False,False,False,True]},
             
             "backend_dict":
             {"n_class":50,
              "bert_config":None, 
              "recurrent_units":2}} # pass None to deactivate

In [ ]:
class CRNNSA(nn.Module):
    
    """
    TODO: explore whether "rec_unit -> self_att -> rec_unit"
    would have worked better.
    """
    
    # BERT-based Convolutional Recurrent Neural Network
    # Code adopted from https://github.com/minzwon/sota-music-tagging-models/
    def __init__(self, main_dict=None, backend=None, frontend=None):
        super(CRNNSA, self).__init__()
        

        if main_dict is not None:
            self.frontend = Frontend_mine(main_dict["frontend_dict"])
            self.backend = Backend2(main_dict)
        else:
            self.frontend = frontend
            self.backend = backend


    def forward(self, spec):
        
        x = self.backend(self.frontend(spec))
        
        return x

In [ ]:
crnnsa = CRNNSA(main_dict)
print(crnnsa)

In [11]:
input_length = 5
input_length = int(input_length*16000/256)

In [12]:
spec = torch.FloatTensor(np.load("/import/c4dm-datasets/rmri_self_att/msd/1/1/113801.npy")[np.newaxis,
                                                                                           np.newaxis,
                                                                                           :,
                                                                                           :input_length])
spec.shape

torch.Size([1, 1, 96, 312])

In [ ]:
crnnsa(spec).shape

In [2]:
dataLoader = get_DataLoader(batch_size=1, input_length=5, mode="valid", num_workers=6)

In [9]:
ctr = 0
for x,y in dataLoader:
    print(x.shape)
    print(y.shape)
    ctr+=1
    if ctr==10:
        break

torch.Size([1, 6, 96, 312])
torch.Size([1, 50])
torch.Size([1, 6, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])
torch.Size([1, 6, 96, 312])
torch.Size([1, 50])
torch.Size([1, 6, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])
torch.Size([1, 12, 96, 312])
torch.Size([1, 50])


In [ ]:
print(crnnsa.frontend(x).shape)
print(crnnsa(x).shape)

In [ ]:
whole_spec = np.load("/import/c4dm-datasets/rmri_self_att/msd/1/1/113801.npy")

n_chunks = whole_spec.shape[1] // input_length
spec = np.zeros((n_chunks,whole_spec.shape[0],input_length))
for i in range(n_chunks):
    spec[i]=whole_spec[:,i*input_length:(i+1)*input_length]

In [ ]:
spec.shape

In [ ]:
whole_spec.shape